In [1]:
#Torch imports
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

#Misc imports
import os
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

In [15]:
#Function to convert string to one hot encoding
def to_onehot(data):
    ascii_list = []
    for songs in data:
        for chars in songs:
            ascii_list.append(ord(chars))
    onehot_vals = np.eye(128)[ascii_list]
    return onehot_vals, np.array(ascii_list)

In [ ]:
#data is a list of list of chars, each string list is a song
#onehot_ascii output are shifted up by 1, need to figure out what to do with 0 array labels
def onehot_songs(data, sequence_length):
    max_num_chars = 0
    temp_list = []
    onehot_ascii = []
    onehot_songs = []
    
    #List of songs in ascii format
    for song in data:
        if len(song)>max_num_chars:
            max_num_chars = len(song) #Finds longest song length
        for chars in song:
            temp_list.append(ord(chars))
        onehot_ascii.append(temp_list)
        temp_list = []
        
    #To make divisible by sequence_length per batch
    max_num_chars = sequence_length-(max_num_chars%sequence_length)+max_num_chars
    codding = np.append(np.zeros((1,128)),np.eye(128),0)    
    
    for i, ascii_song in enumerate(onehot_ascii):
        ascii_song = np.array(ascii_song)+1 #since making first row of eye matrix all 0s
        needed_0s = max_num_chars-len(ascii_song)
        onehot_ascii[i] = np.pad(ascii_song,(0,needed_0s),'constant',constant_values=0)
        onehot_songs.append(codding[onehot_ascii[i]])
        
    return onehot_ascii, onehot_songs

In [3]:
file = open("./Data/input.txt")
text = file.readlines()
file.close()

text_array = np.asarray(text)

#Creates list of each song
indicies = np.where(text_array == '<start>\n') #Location of where each abc file starts
data = []
for i in range(len(indicies[0])):
    if i+1 == len(indicies[0]):
        #For the last abc file
        abc = text_array[indicies[0][i]:]
    else:
        abc = text_array[indicies[0][i]:indicies[0][i+1]]
    data.append(''.join(abc))

#print(data[0])
#print(data[1123])

In [7]:
#80 - 20 split on data -> training and validation
#Constants
train_len = int(np.floor(len(data)*0.8))
validation_len = len(data) - train_len

print(train_len)
print(validation_len)

np.random.seed(0)
#Each index references to a single song
indxs = np.asarray(range(len(data)))
print(indxs)
np.random.shuffle(indxs)
print(indxs)

train_data = (np.asarray(data))[indxs[0:train_len]]
validation_data = (np.asarray(data))[indxs[train_len:]]
print(len(train_data))
print(len(validation_data))

899
225
[   0    1    2 ... 1121 1122 1123]
[ 752  893 1050 ...  835  559  684]
899
225


In [92]:
#Visualizing to_onehot
b=5
a,b = to_onehot('abc')
print(np.argmax(a,1))
print(type(np.argmax(a,1)))
print(b)
print(type(b))

[97 98 99]
<class 'numpy.ndarray'>
[97 98 99]
<class 'numpy.ndarray'>


In [ ]:
#Visualizing onehot_songs
train_data[0][:]
#len(train_data[0:2])
a,b = onehot_songs(train_data[0:2],25)
print(b[0][374])
print(a[0])
print(len(a[0]))
print(a[1])
print(len(a[1]))

In [19]:
#Data

#combines all the songs and makes a single string, converted to ascii and onehot
onehot_train, train_labels = to_onehot(train_data)
onehot_validation, validation_labels = to_onehot(validation_data)
#print(train_labels[0])
#print(onehot_train[0])

#or using onehot song encoddings
onehot_train, train_labels = onehot_songs(train_data)
onehot_validation, validation_labels = onehot_songs(validation_data)
#print(train_labels[0])
#print(onehot_train[0])

In [ ]:
#LSTM constants
input_size = 128 #num_classes = 128
hidden_size = 128
num_layers = 1
batch_first = True
batch_size = 1
sequence_length = 25


run first in single batches, each batch has 25 chars per(sequence length 25), just run it in order of the train data

In [ ]:
class lstm_rnn(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers=1,batch_size=1,
                 sequence_length=25,batch_first = True):
        super(lstm_rnn, self).__init__()
        #Constants
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        
        #Initializing model
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first = batch_first)
        
    def forward(self,data,initial):
        #data = data.view(self.batch_size, self.sequence_length, self.input_size)
        output,hidden = self.lstm(data,initial)
        return output,hidden
        
    def init_hidden(self,zero=1):
        if zero == 0:
            #Random initialization
            initial_hidden = autograd.Variable(torch.randn(self.num_layers,self.batch_size,self.hidden_size))
        else:
            #Zero initialization
            initial_hidden = autograd.Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_size))
        return initial_hidden
    
    def init_cell(self,zero=1):
        if zero == 0:
            #Random initialization
            initial_cell = autograd.Variable(torch.randn(self.num_layers,self.batch_size,self.hidden_size))
        else:
            #Zero initialization
            initial_cell = autograd.Variable(torch.zeros(self.num_layers,self.batch_size,self.hidden_size))
        return initial_cell
    

In [ ]:
lstm = lstm_rnn(input_size,hidden_size,num_layers,batch_size,
                sequence_length,batch_first = batch_first)
print(lstm.input_size)
print(lstm.hidden_size)
print(lstm.batch_size)
print(lstm.sequence_length)
print(lstm.num_layers)
print(lstm)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lstm.parameters(), lr=0.1)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [ ]:
#Creates batch indicies, +1 since arange is [,), exclusive last element
batch_indicies = np.arange(0,len(onehot_train)-sequence_length+1,sequence_length)
print(len(batch_indicies))

In [ ]:
h_0 = lstm.init_hidden()
c_0 = lstm.init_cell()
states = (h_0,c_0)
loss = []
accuracy = []

#CAN ADD LINEAR LAYER AND BATCH PROCESSING 
for epoch in range(5):
    for indx in batch_indicies:
        optimizer.zero_grad()
        temp_data = onehot_train[indx:indx+25]
        temp_data = torch.from_numpy(temp_data).float()
        temp_data = autograd.Variable(temp_data.view(batch_size,sequence_length,input_size))
    
        #output is of whole sequence, states hidden is same as last element of output, also
        #has cell hidden state
        output, states = lstm(temp_data,states)
        
        h_0 = autograd.Variable(states[0].data, requires_grad=True)
        c_0 = autograd.Variable(states[1].data, requires_grad=True)
        states = (h_0,c_0)
        #hidden = Variable(hidden.data, requires_grad=True)

        
        #Add 1 since we are using the next letter as the label
        labels = autograd.Variable(torch.LongTensor(train_labels[indx+1:indx+25+1]))
        
        #Later try adding a linear layer here
        
        loss_temp = criterion(output.view(sequence_length,input_size),labels)
        loss.append(loss_temp.data)
        loss_temp.backward()
        optimizer.step()
        
        vals,max_indxs = (output.data).max(2)
        acc_temp = (max_indxs==(labels.data)).sum()
        accuracy.append(acc_temp/sequence_length)
        #print(loss)
        #print(accuracy)
    print(epoch)
        

        

In [ ]:
plt.plot(loss)

In [ ]:
#write a function for this later 
primer = "<start>\n"
start_token = []
for chars in primer:
    start_token.append(ord(chars))
    
start_token = np.eye(128)[start_token]
start_token = torch.from_numpy(start_token).float()
start_token = autograd.Variable(start_token.view(1,8,input_size))
#print(start_token.argmax(1))

end = "<end>"
end_token = []
for chars in end:
    end_token.append(ord(chars))
    
end_token = np.eye(128)[end_token]

done = 1

#Generating music
h_0 = lstm.init_hidden()
c_0 = lstm.init_cell()
states = (h_0,c_0)
chars = []

#Each loop generates 1 char
for i in range(100):
    output,states = lstm(start_token,states)
    
    h_0 = autograd.Variable(states[0].data, requires_grad=True)
    c_0 = autograd.Variable(states[1].data, requires_grad=True)
    states = (h_0,c_0)
    
    #Currently I am taking the max of the hidden state as the letter that it
    #produces, the TA on piazza said that it is not ideal to take max,
    #can try implementing what he says later
    val,indx = states[0].data.max(2)
    chars.append(indx)
    
    start_token = torch.from_numpy(np.eye(128)[indx]).float()
    start_token = autograd.Variable(start_token.view(1,1,input_size))
    
    #Need to implement a check for "<end>", generation should stop once
    #"<end>" is produced

In [ ]:
#Converting back to strings
outputs = []
for letter in chars:
    outputs.append(chr(letter))

print(chars)
print(outputs)
